In [ ]:
import kessler
from kessler import EventDataset
from kessler.nn import LSTMPredictor
from kessler.data import kelvins_to_event_dataset

import pandas as pd

# Data Loading

Kessler accepts CDMs either in KVN format or as pandas dataframes. We hereby show a pandas dataframe loading example:

In [ ]:
#As an example, we first show the case in which the data comes from the Kelvins competition.
#For this, we built a specific converter that takes care of the conversion from Kelvins format
#to standard CDM format (the data can be downloaded at https://kelvins.esa.int/collision-avoidance-challenge/data/):
file_name = 'path_to_csv/train_data.csv'
events = kelvins_to_event_dataset(file_name, drop_features=['c_rcs_estimate', 't_rcs_estimate'], num_events=200) #we use only 200 events

In [ ]:
#Instead, this is a generic real CDM data loader that should parse your Pandas (uncomment the following lines if needed):
#file_name = 'path_to_csv/file.csv'

#df=pd.read_csv(file_name)
#events = EventDataset.from_pandas(df)

# Descriptive Statistics

In [ ]:
#Descriptive statistics of the event:
kessler_stats = events.to_dataframe().describe()
print(kessler_stats)


# LSTM Training

In [ ]:
#We only use features with numeric content for the training
#nn_features is a list of the feature names taken into account for the training:
#it can be edited in case more features want to be added or removed
nn_features = events.common_features(only_numeric=True)
print(nn_features)

In [ ]:
# Split data into a test set (5% of the total number of events)
len_test_set=int(0.05*len(events))
print('Test data:', len_test_set)
events_test=events[-len_test_set:]
print(events_test)

# The rest of the data will be used for training and validation
print('Training and validation data:', len(events)-len_test_set)
events_train_and_val=events[:-len_test_set]
print(events_train_and_val)

In [ ]:
# Create an LSTM predictor, specialized to the nn_features we extracted above
model = LSTMPredictor(features=nn_features)

# Start training
model.learn(events_train_and_val, 
            epochs=3, # Number of epochs (one epoch is one full pass through the training dataset)
            lr=1e-4, # Learning rate, can decrease it if training diverges
            batch_size=16, # Minibatch size, can be decreased if there are issues with memory use
            device='cpu', # Can be 'cuda' if there is a GPU available
            valid_proportion=0.15, # Proportion of the data to use as a validation set internally
            num_workers=4, # Number of multithreaded dataloader workers, 4 is good for performance, but if there are any issues or errors, please try num_workers=1 as this solves issues with PyTorch most of the time
            event_samples_for_stats=1000) # Number of events to use to compute NN normalization factors, have this number as big as possible (and at least a few thousands)

In [ ]:
#Save the model to a file after training:
model.save(file_name="LSTM_20epochs_lr10-4_batchsize16")

In [ ]:
#NN loss plotted to a file:
model.plot_loss(file_name='plot_loss.pdf')

In [ ]:
#we show an example CDM from the set:
events_train_and_val[0][0]

In [ ]:
#we take a single event, we remove the last CDM and try to predict it
event=events_test[3]
event_len = len(event)
print(event)
event_beginning = event[0:event_len-1]
print(event_beginning)
event_evolution = model.predict_event(event_beginning, num_samples=100, max_length=14)

In [ ]:
#We plot the prediction in red:
axs = event_evolution.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], return_axs=True, linewidth=0.1, color='red', alpha=0.33, label='Prediction')
#and the ground truth value in blue:
event.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], axs=axs, label='Real', legend=True)

In [ ]:
#we now plot the uncertainty prediction for all the covariance matrix elements of both OBJECT1 and OBJECT2:
axs = event_evolution.plot_uncertainty(return_axs=True, linewidth=0.5, label='Prediction', alpha=0.5, color='red', legend=True, diagonal=False)
event.plot_uncertainty(axs=axs, label='Real', diagonal=False)

# Plotting loop over all the events & CDMs
You can here customize the features to be plotted: we use relative speed, miss distance, and a covariance value:

In [ ]:
#we loop over the test set events:
predict_full_event=False
for i in range(0,len(events_test)):
    event=events_test[i]
    len_ev=len(event)
    for j in range(1,len_ev):
        #print(j)
        if predict_full_event:
            event_evolution = model.predict_event(event[0:j],num_samples=10)
        else:
            event_evolution = model.predict_event_step(event[0:j],num_samples=10)

        #we plot the features (ground truth & prediction)
        axs_1 = event_evolution.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], return_axs=True, linewidth=0.1, color='red', alpha=0.33, label='Prediction')
        event.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], axs=axs_1, label='Real', legend=True,file_name=f'features_event_{i}_cdm_{j}.pdf')
        #we plot the uncertainties (ground truth & prediction)
        axs_2 = event_evolution.plot_uncertainty(return_axs=True, linewidth=0.5, label='Prediction', alpha=0.5, color='red', legend=True, diagonal=False)
        event.plot_uncertainty(axs=axs_2, label='Real', diagonal=False, file_name=f'uncertainties_event_{i}_cdm_{j}.pdf')

# Training set test
We check if the model is able to predict the CDMs on the training set

In [ ]:

#we loop over some training set events, to check the NN performances:
num_events=10
for i in range(0,num_events):
    event=events_train_and_val[i]
    len_ev=len(event)
    for j in range(1,len_ev):
        print(j)
        event_evolution = model.predict_event(event[0:j],num_samples=10)
        #we plot the features (ground truth & prediction)
        axs_1 = event_evolution.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], return_axs=True, linewidth=0.1, color='red', alpha=0.33, label='Prediction')
        event.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], axs=axs_1, label='Real', legend=True,file_name=f'training_set_features_event_{i}_cdm_{j}.pdf')
        #we plot the uncertainties (ground truth & prediction)
        axs_2 = event_evolution.plot_uncertainty(return_axs=True, linewidth=0.5, label='Prediction', alpha=0.5, color='red', legend=True, diagonal=False)
        event.plot_uncertainty(axs=axs_2, label='Real', diagonal=False, file_name=f'training_set_uncertainties_event_{i}_cdm_{j}.pdf')